In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 200 kB 45.2 MB/s 
     |████████████████████████████████| 199 kB 14.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=787ada8bfd99bda3249518c99ffc61c822c81f1902d5b7da80be8e5bf8e053a4
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mytree').getOrCreate()
from pyspark.ml import Pipeline
from pyspark.ml.classification import (RandomForestClassifier,GBTClassifier,DecisionTreeClassifier)

In [4]:
data = spark.read.format('libsvm').load('/content/drive/MyDrive/Colab Notebooks/data/sample_libsvm_data.txt')
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [5]:
train_data,test_data = data.randomSplit([0.7,0.3])
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gbt = GBTClassifier()

In [7]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [8]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [9]:
rfc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[121,122,123...|   [19.0,1.0]|[0.95,0.05]|       0.0|
|  0.0|(692,[122,123,124...|   [19.0,1.0]|[0.95,0.05]|       0.0|
|  0.0|(692,[124,125,126...|   [20.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [18.0,2.0]|  [0.9,0.1]|       0.0|
|  0.0|(692,[151,152,153...|   [20.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[154,155,156...|   [15.0,5.0]|[0.75,0.25]|       0.0|
|  0.0|(692,[154,155,156...|   [13.0,7.0]|[0.65,0.35]|       0.0|
|  1.0|(692,[97,98,99,12...|   [4.0,16.0]|  [0.2,0.8]|       1.0|
|  1.0|(692,[124,125,126...|   [0.0,20.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[125,126,127...|   [1.0,19.0]|[0.05,0.95]|       1.0|
|  1.0|(692,[126,127,128...|   [0.0,20.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[126,127,128...|   [0.0,20.0]|  [0.0,1.0]|       1.0|
|  1.0|(69

In [10]:
gbt_preds.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[121,122,123...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[122,123,124...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[124,125,126...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[151,152,153...|[1.05459799499933...|[0.89179377260678...|       0.0|
|  0.0|(692,[154,155,156...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[154,155,156...|[-1.0148951408703...|[0.11611044347130...|       1.0|
|  1.0|(692,[97,98,99,12...|[-1.5435020027249...|[0.04364652142729...|       1.0|
|  1.0|(692,[124,125,126...|[-1.5435020027249...|[0.04364652142729...|       1.0|
|  1.0|(692,[125

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')
print('DTC_ACCURACY:')
acc_eval.evaluate(dtc_preds)

DTC_ACCURACY:


0.9545454545454546

In [13]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')
print('RFC_ACCURACY:')
acc_eval.evaluate(rfc_preds)

RFC_ACCURACY:


1.0

In [14]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')
print('GBT_ACCURACY:')
acc_eval.evaluate(gbt_preds)

GBT_ACCURACY:


0.9545454545454546

In [15]:
rfc_model.featureImportances

SparseVector(692, {121: 0.0026, 147: 0.0029, 158: 0.0018, 186: 0.0025, 207: 0.0144, 235: 0.0247, 237: 0.0017, 263: 0.0406, 273: 0.0556, 291: 0.0134, 301: 0.0089, 324: 0.0052, 344: 0.0382, 346: 0.0068, 350: 0.0779, 351: 0.0021, 356: 0.0266, 359: 0.0171, 360: 0.0032, 377: 0.0474, 378: 0.0129, 380: 0.0062, 406: 0.1, 408: 0.0026, 426: 0.0114, 435: 0.006, 439: 0.0237, 441: 0.0092, 442: 0.0178, 443: 0.0166, 456: 0.0341, 462: 0.0053, 466: 0.0217, 484: 0.0411, 485: 0.0045, 489: 0.0478, 490: 0.1113, 512: 0.0118, 540: 0.0818, 544: 0.0027, 548: 0.0025, 552: 0.0274, 607: 0.0058, 623: 0.0004, 651: 0.0015})